# Node2Vec

In network science, there’s a useful tool called a random walk. It is a simulation of traffic on a network, where tiny “agents” move from node to node according to some probabilities set in the network. Borrowing from a passage on the Wikipedia article,

“…one can imagine a person walking randomly around a city. The city is effectively infinite and arranged in a square grid of sidewalks. At every intersection, the person randomly chooses one of the four possible routes (including the one originally traveled from)… Will the person ever get back to the original starting point of the walk?”

node2vec learns representations of nodes in a graph through the application of the word2vec model on sequences of nodes sampled through random walks. The innovation brought by node2vec is the **definition of a random walk exploration** that is flexible and adaptable to the diversity of connectivity patterns that a network may present. 

In order to generate our corpus from the input graph, let’s think about a corpus as a group of directed acyclic graphs, with a maximum out degree of 1. If we think about it this is a perfect representation for a text sentence, where each word in the sentence is a node and it points on the next word in the sentence.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "Hqiqs.gif")

### Mathematics of Node2Vec

http://www.lirmm.fr/~sau/JCALM/Josep.pdf

https://websites.math.leidenuniv.nl/probability/lecturenotes/RandomWalks.pdf

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "embedding.png")

Node2vec’s sampling strategy, accepts 4 arguments:

— Number of walks: Number of random walks to be generated from each node in the graph

— Walk length: How many nodes are in each random walk

— P: Return hyperparameter

— Q: Inout hyperaprameter

and also the standard skip-gram parameters (context window size, number of iterations etc.)


The algorithm for the random walk generation will go over each node in the graph and will generate <number of walks> random walks, of length <walk length>.

Consider you are on the random walk, and have just transitioned from node <t> to node <v> in the following diagram

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "random_walks.gif")

### Implementation in Python

In [3]:
import warnings
from text_unidecode import unidecode
from collections import deque
warnings.filterwarnings('ignore')


import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from node2vec import Node2Vec

In [ ]:
import os
import pandas as pd

path="C:/Users/adsieg/Desktop/node2vec"
os.chdir(path)

news = pd.read_csv('news.csv')
text_total = news.description 
text_total = text_total.reset_index(drop=True)

In [ ]:
text_total.head()

### Cleaning Text

In [ ]:
# import basic libraries
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
stemmer = PorterStemmer()

# function to clean text
def review_to_words(raw_review):
    
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 
    
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    
    # 3. Remove Stopwords. In Python, searching a set is much faster than searching a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]  #returns a list 

    # 5. Stem words. Need to define porter stemmer above
    singles = [stemmer.stem(word) for word in meaningful_words]
    
    # 7. remove remaining tokens that are not alphabetic
    singles = [word for word in singles if word.isalpha()]
    
    # 8. Join the words back into one string separated by space, and return the result.
    return( " ".join( singles ))

In [ ]:
# apply it to our text data 
# dataset is named wine_data and the text are in the column "wmn"
processed_wmn = [ review_to_words(str(text)) for text in text_total]

In [ ]:
processed_wmn[:5]

In [ ]:
processed_words = " ".join(processed_wmn).split()

### Transform sentences into pair of words

In [ ]:
def transform_sentences_to_pair_of_words(sentences):
    list_of_pair_of_words = []
    for i in range(len(sentences)):
        buffer_pair_of_words = (sentences[i-1],sentences[i])
        list_of_pair_of_words.append(buffer_pair_of_words)
    del list_of_pair_of_words[0]
    return list_of_pair_of_words

sentences_to_pair = transform_sentences_to_pair_of_words(processed_words)

In [ ]:
sentences_to_pair = sentences_to_pair[:10000]

In [ ]:
sentences_to_pair

### From pair of words to Graph network of words

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

G = nx.DiGraph()

In [ ]:
G.add_edges_from(sentences_to_pair)

In [ ]:
G = G.to_undirected()

In [ ]:
type(G)

In [ ]:
import matplotlib

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
import networkx as nx
nx.draw_networkx(G)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

print("Nodes of graph: ")
print(G.nodes())
print("Edges of graph: ")
print(G.edges())
nx.draw(G)
plt.show()

### Node2Vec

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor
from time import sleep

In [ ]:
node2vec = Node2Vec(G, dimensions=20, walk_length=16, num_walks=100, workers=2)

In [ ]:
model = node2vec.fit(window=10, min_count=1)

In [ ]:
from gensim.models import Word2Vec

model.wv.save_word2vec_format('node2vec_embeddings')
model.save('node2vec_model')

print("Model Saved")

In [ ]:
from gensim.models import Word2Vec
model= Word2Vec.load("C:/Users/adsieg/Desktop/node2vec/node2vec_model")

In [ ]:
for node in model.wv.most_similar('casino'):
    # Show only players
    if len(node) > 3:
        print(node)

In [ ]:
def main():
    # Test to make sure concurrent map is working
    with concurrent.futures.ProcessPoolExecutor() as executor = concurrent.futures.ThreadPoolExecutor(maxworkers=2)
        node2vec = Node2Vec(G, dimensions=20, walk_length=16, num_walks=100, workers=2)

if __name__ == '__main__':
    main()

In [ ]:
model.wv.vocab

### TSNE draw words

In [ ]:
player_nodes = [x for x in model.wv.vocab if len(x) > 3]
embeddings = np.array([model.wv[x] for x in player_nodes])

In [ ]:
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)